In [ ]:
from unsloth import FastLanguageModel
import torch
import pandas as pd

In [ ]:
max_seq_length = 4096  # Choose any! We auto support RoPE Scaling internally!
dtype = None  # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
# Use 4bit quantization to reduce memory usage. Can be False.
load_in_4bit = True

In [ ]:
import pandas as pd
df = pd.read_csv('./datasets/gemini_generated/gemini_dataset_v0.csv')
df2 = pd.read_csv('./datasets/gemma10000.csv')
df = pd.concat([df, df2])

In [ ]:
df = pd.concat([df, df2])

In [ ]:
df = pd.read_csv('./datasets/datasets_with_cot(1).csv')
df = df.query('cot == cot')

In [ ]:
remove_ids = []
for idx, row in df.iterrows():
    try:
        row['rewritten_text'].split('\n\n')[1]
    except:
        remove_ids.append(idx)

In [ ]:
df = df.drop(remove_ids)

In [ ]:
df['rewritten_text'] = df['rewritten_text'].apply(lambda x: x.split('\n\n')[1])

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/gemma-7b-it-bnb-4bit",  # Instruct version of Gemma 7b
    "unsloth/gemma-2b-bnb-4bit",
    "unsloth/gemma-2b-it-bnb-4bit",  # Instruct version of Gemma 2b
]  # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="./gemma/7b-it/",  # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=64,  # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj",],
    lora_alpha=16,
    lora_dropout=0.05,  # Supports any, but = 0 is optimized
    bias="none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing=True,
    random_state=3407,
    use_rslora=False,  # We support rank stabilized LoRA
    loftq_config=None,  # And LoftQ
)

In [ ]:
# import pandas as pd
# df1 = pd.read_csv('./datasets/gemma10000.csv')
# df2 = pd.read_csv('./plots2k.csv')
# df2 = df2.drop(1745).reset_index(drop = True)
# df2.columns = ['original_text','rewritten_text','rewrite_prompt']
# df1 = df1[df2.columns]
# df = pd.concat([df1, df2]).sample(frac = 1).reset_index(drop = True)
# df.head()

In [ ]:
# def formatting_prompts_func(examples):
#     USER_CHAT_TEMPLATE = "<start_of_turn>user\n{}<end_of_turn>\n<start_of_turn>model\n{}<end_of_turn>"
#     inst = """You will be given original text and rewritten text.
# Analyzing the changes in style, theme, etc., your task is to recovery the prompt which LLM used to convert the original text.
# **Original Text**:
# {}
# **Rewritten Text**
# {}
# """
#     original_texts = examples["original_text"]
#     rewrite_prompts       = examples["rewrite_prompt"]
#     rewritten_texts      = examples["rewritten_text"]
#     texts = []
#     EOS_TOKEN = tokenizer.eos_token
#     for original_text, rewritten_text, rewrite_prompt in zip(original_texts, rewritten_texts, rewrite_prompts):
#         # Must add EOS_TOKEN, otherwise your generation will go on forever!
#         inst = inst.format(original_text[:750],rewritten_text[:750])
#         text = USER_CHAT_TEMPLATE.format(inst, rewrite_prompt) + EOS_TOKEN
#         texts.append(text)
#     return { "text" : texts}

In [ ]:
from sklearn.model_selection import train_test_split
prompts = list(df['rewrite_prompt'].unique())
train_prompts, test_prompts = train_test_split(
    prompts, test_size=0.1, random_state=42)

In [ ]:
train_df = df[df['rewrite_prompt'].isin(train_prompts)]
test_df = df[~df['rewrite_prompt'].isin(train_prompts)]

In [ ]:
from datasets import Dataset
train_data = Dataset.from_pandas(train_df)
test_data = Dataset.from_pandas(test_df)

In [ ]:
# from datasets import Dataset
# ds = Dataset.from_pandas(df)
# ds_split = ds.train_test_split(test_size=0.1, seed = 777)
# train_data = ds_split["train"]
# test_data = ds_split["test"]

In [ ]:
import random



def truncate_txt(text, length):

    text_list = text.split()


    if len(text_list) <= length:
        return text


    return " ".join(text_list[:length])



def formatting_prompts_func(examples):

    USER_CHAT_TEMPLATE = "<start_of_turn>user\n{}<end_of_turn>\n<start_of_turn>model\n{}<end_of_turn>"

#         inst = """You'll be given an original text and a rewritten text generated by an LLM.

# Analyze the stylistic changes in the rewritten version and identify the likely prompt that led to those changes.

# Notice only output the prompt.

# Here's what to focus on:

# Shifts in Style: Look for changes in:

# -Genre: (sci-fi, fantasy, historical fiction, etc.)

# -Tone: (serious, humorous, conversational, etc.)

# -Vocabulary: (formal vs. informal, technical vs. simple)

# -Sentence Structure: (short and direct vs. flowing and complex)

# Literary References: Consider if the rewritten style echoes a specific author or literary period (e.g., Shakespearean, Hemingway-esque).


# **Original Text**:

# {}


# **Rewritten Text**

# {}"""

    inst = """You'll be given an original text and a rewritten text generated by an LLM. 

Analyze the changes in the rewritten version and infer the likely prompt that led to those changes. 

Provide a detailed explanation of how you arrived at your inference step by step.


**Original Text**:

{}


**Rewritten Text**

{}


You should response in the following format:

**Inferred Promp**: ...


**Chain of Thoughts**: ..."""

    EOS_TOKEN = tokenizer.eos_token

    # while True:

    #     idx = random.randint(0, len(train_data) - 1)

    #     if train_data["original_text"][idx] != examples["original_text"] or train_data["rewrite_prompt"][idx] != examples["rewrite_prompt"]:

    #         break

    # e_o_t = truncate_txt(train_data["original_text"][idx],100)

    # e_r_t = truncate_txt(train_data["rewritten_text"][idx], 100)

    # e_ans = train_data["rewrite_prompt"][idx]

    ot = truncate_txt(examples["original_text"], 400)

    rt = truncate_txt(examples["rewritten_text"], 400)


    # inst = inst.format(e_o_t, e_r_t, e_ans, ot, rt)

    inst = inst.format(ot, rt)

    rewrite_prompts = examples["cot"]

    text = USER_CHAT_TEMPLATE.format(inst, rewrite_prompts)

    text += EOS_TOKEN

    return {"text": text}

In [ ]:
train_data = train_data.map(formatting_prompts_func)
test_data = test_data.map(formatting_prompts_func)

In [ ]:
print(test_data['text'][0])

In [ ]:
# from sentence_transformers import SentenceTransformer
# sentence_model = SentenceTransformer('./sentence-t5-base').cpu()
# from numpy import dot
# from numpy.linalg import norm
# def computeSharpenedCosineSimilarity(sent1, sent2):
#     embeddings = sentence_model.encode([sent1, sent2])
#     cos_sim = dot(embeddings[0], embeddings[1]) / (norm(embeddings[0]) * norm(embeddings[1]))
#     return cos_sim ** 3

# def compute_metrics(p):
#     preds, labels = p

#     # Remove ignored index (special tokens)
#     preds = [
#         [p for (p, l) in zip(prediction, label) if l != -100]
#         for prediction, label in zip(preds, labels)
#     ]
#     true_labels = [
#         [l for (p, l) in zip(prediction, label) if l != -100]
#         for prediction, label in zip(preds, labels)
#     ]
#     print(decoded_preds)
#     print(decoded_labels)
#     decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True, device="cpu")
#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True, device="cpu")

#     res = 0
#     for s1, s2 in zip(decoded_preds, decoded_labels):
#         res += computeSharpenedCosineSimilarity(s1, s2)
#     res /= len(decoded_preds)

#     results = {
#         'SCS': res
#     }
#     return results

In [ ]:
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from transformers import TrainingArguments
response_template = "<start_of_turn>model\n"
args = TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    per_device_eval_batch_size=1,
    warmup_steps=0.05,
    num_train_epochs=15,
    learning_rate=5e-5,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    logging_steps=100,
    evaluation_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="cosine",
    seed=3407,
    output_dir="./outputs/gemini_data/v7_cot_5k_remove_prompt",
)
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_data,
    eval_dataset=test_data,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,  # Can make training 5x faster for short sequences.
    # neftune_noise_alpha=5,
    args=args,
    # compute_metrics = compute_metrics,
    # data_collator=DataCollatorForCompletionOnlyLM(response_template=response_template,tokenizer=tokenizer, mlm=False),
)

In [ ]:
trainer_stats = trainer.train()

In [ ]:
# torch.save(train_data, "tran_data.pkl")

In [ ]:
# torch.save(test_data, "test_data.pkl")

In [ ]:
# trainer.model.save_pretrained("./output/test_lora_model")

In [ ]:
# from transformers import AutoModelForCausalLM, AutoTokenizer
# # Merge the model with LoRA weights
# base_model = AutoModelForCausalLM.from_pretrained(
#     "./gemma/gemma-7b-it-bnb-4bit/",
#     load_in_4bit = True,
#     torch_dtype=torch.float16,
#     device_map={"": 0},
# )

In [ ]:
# from peft import LoraConfig, PeftModel
# merged_model= PeftModel.from_pretrained(base_model, './output/test_lora_model')

In [ ]:
# merged_model= merged_model.merge_and_unload()

In [ ]:
# merged_model.save_pretrained("./output/merged_model",safe_serialization=True)

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained("./gemma/gemma-7b-it-bnb-4bit/")
# tokenizer.save_pretrained("./output/merged_model")

In [ ]:
test_data[0]['text'].split('<start_of_turn>model\n')[
    0] + '<start_of_turn>model\n'

In [ ]:
FastLanguageModel.for_inference(model)  # Enable native 2x faster inference
inputs = tokenizer(
    [
        test_data[12]['text'].split('<start_of_turn>model\n')[
            0] + '<start_of_turn>model\n'
    ], return_tensors="pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens=100, use_cache=True)
tokenizer.batch_decode(outputs)

In [ ]:
test_data[12]['rewrite_prompt']

In [ ]:
model.config.forced_eos_token_id

In [ ]:
import pandas as pd
import polars as pl
df = pl.read_csv("./datasets/train.csv")
df['original_text'][0]
df['rewritten_text'][0]
df['rewrite_prompt'][0]

In [ ]:
from peft import PeftModel
import torch

from transformers import AutoModelForCausalLM, AutoTokenizer

# Merge the model with LoRA weights

from transformers import BitsAndBytesConfig


bnb_config = BitsAndBytesConfig(

    load_in_4bit=True,

    bnb_4bit_quant_type="nf4",

    bnb_4bit_compute_dtype=torch.bfloat16,

    bnb_4bit_use_double_quant=True,

)

base_model = AutoModelForCausalLM.from_pretrained(

    "./gemma/7b-it/",

    quantization_config=bnb_config,

    device_map="auto"

)

In [ ]:
import os
res_dict = {}

files = os.listdir('./outputs/gemini_data/')

test_data = torch.load('./test_data.pkl')

In [ ]:
from numpy.linalg import norm
from numpy import dot
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("./gemma/7b-it/")

sentence_model = SentenceTransformer('./sentence-t5-base')



def computeSharpenedCosineSimilarity(sent1, sent2):

    embeddings = sentence_model.encode([sent1, sent2])

    cos_sim = dot(embeddings[0], embeddings[1]) / \
        (norm(embeddings[0]) * norm(embeddings[1]))

    return cos_sim ** 3

In [ ]:
test_data[0]['text']

In [ ]:
from tqdm import tqdm
for f in tqdm(files):
    if not f.startswith('checkpoint'):
        continue
    lora_model = PeftModel.from_pretrained(
        base_model,
        f"./outputs/gemini_data/{f}/")
    res = []
    truths = []
    for example in test_data:
        truths.append(example['rewrite_prompt'])
        prompt = example['text'].split('<start_of_turn>model\n')[
            0] + '<start_of_turn>model\n'
        prompt_tokenized = tokenizer(prompt, return_tensors="pt").to("cuda")
        output_tokenized = lora_model.generate(**prompt_tokenized,
                                               max_new_tokens=50,
                                               use_cache=True,
                                               do_sample=True,
                                               temperature=0.8,
                                               )[0]
        # remove prompt from output
        output_tokenized = output_tokenized[len(
            prompt_tokenized["input_ids"][0]):]
        res.append(tokenizer.decode(
            output_tokenized).split('<end_of_turn>')[0])
    acc = 0
    for s1, s2 in zip(res, truths):
        acc += computeSharpenedCosineSimilarity(s1, s2)
    res_dict[f] = acc

In [ ]:
for k in res_dict:
    res_dict[k] = res_dict[k] / len(test_data)

In [ ]:
torch.save(res_dict, "res_dict.pkl")

In [ ]:
res_list = [[k, v] for k, v in res_dict.items()]
res_list.sort(key=lambda x: x[1], reverse=True)
res_list

In [ ]:
acc = 0
for s1, s2 in zip(res, truths):
    acc += computeSharpenedCosineSimilarity(s1, s2)
print(acc / len(res))

In [ ]:
res[17]

In [ ]:
# USER_CHAT_TEMPLATE = """<start_of_turn>user\nYou will be given original text and rewritten text.
# Analyzing the changes in style, theme, etc., your task is to recovery the prompt which LLM used to convert the original text.
# **Original Text**:
# {}
# **Rewritten Text**
# {}<end_of_turn>\n<start_of_turn>model\n"""

In [ ]:
USER_CHAT_TEMPLATE = """<start_of_turn>user
Original Essay:
\"""{}\"""

Rewritten Essay:
\"""{}\"""

Given are 2 essays, the Rewritten essay was created from the Original essay using the google Gemma model.
You are trying to understand how the original essay was transformed into a new version.
Analyzing the changes in style, theme, etc., please come up with a prompt that must have been used to guide the transformation from the original to the rewritten essay.
Start directly with the prompt, that's all I need. Output should be only line ONLY.<end_of_turn>\n<start_of_turn>model\n
"""

In [ ]:
import pandas as pd
df = pd.read_csv('./datasets/gemma100.csv')
df.head()

In [ ]:
from numpy.linalg import norm
from numpy import dot
from sentence_transformers import SentenceTransformer

sentence_model = SentenceTransformer('./sentence-t5-base')



def computeSharpenedCosineSimilarity(sent1, sent2):

    embeddings = sentence_model.encode([sent1, sent2])

    cos_sim = dot(embeddings[0], embeddings[1]) / \
        (norm(embeddings[0]) * norm(embeddings[1]))

    return cos_sim ** 3

In [ ]:
lora_model = PeftModel.from_pretrained(
    base_model,
    "./outputs/gemini_data/v1/checkpoint-1200/")

In [ ]:
from tqdm import tqdm
ots = list(df['original_text'])
rts = list(df['rewritten_text'])
prompts_all = []
for idx, (ot, rt) in enumerate(zip(ots, rts)):
    prompts_all.append(USER_CHAT_TEMPLATE.format(ot[:1500], rt[:1500]))
res = []
for prompt in tqdm(prompts_all):
    prompt_tokenized = tokenizer(prompt, return_tensors="pt").to("cuda")
    output_tokenized = lora_model.generate(**prompt_tokenized,
                                           max_new_tokens=100,
                                           use_cache=True,
                                           do_sample=False,
                                           temperature=0.8,
                                           )[0]

    # remove prompt from output
    output_tokenized = output_tokenized[len(prompt_tokenized["input_ids"][0]):]
    res.append(tokenizer.decode(output_tokenized).split('<end_of_turn>')[0])
acc = 0
for s1, s2 in zip(res, list(df["rewrite_prompt"])):
    acc += computeSharpenedCosineSimilarity(s1, s2)
print(acc / len(res))

In [ ]:
prompts = pd.read_csv('./gemini_prompts.csv')

In [ ]:
ps = set(prompts['rewrite_prompt'])

In [ ]:
for p in ps:
    if "news article" in p:
        print(p)

In [ ]:
res[0]

In [ ]:
res

In [ ]:
inputs = tokenizer(
    [
        "<start_of_turn>user\nYou will be given original text and rewritten text. \nAnalyzing the changes in style, theme, etc., your task is to recovery the prompt which LLM used to convert the original text.\n**Original Text**:\nCTVNews.ca Staff She may not be strategizing offensive plays or coaching players on their tackles, but Catherine Raiche is an integral part of the team. The 28-year-old former corporate and tax lawyer has spent the past six months working as the Assistant General Manager for the Montreal Alouettes. She is also the only woman in football operations for the Canadian Football League. As part of her new position, Raiche handles players’ contracts, manages the salary cap and helps out with scouting new talent. “She is a task master,” the team’s General Manager Kavis Reed told CTV News. “You give her\n**Rewritten Text**\n## The Catherine Raiche Collection: Breaking Barriers in Football Operations\n\nCatherine Raiche is a force to be reckoned with, shattering glass ceilings and leaving her mark on the world of professional football. Her journey from corporate lawyer to assistant general manager for the Montreal Alouettes is a testament to her unwavering dedication and fierce determination.\n\nRaiche's expertise lies beyond simply managing contracts and budgets. She possesses a profound understanding of player management, talent acquisition, and operations. Her meticulous attention to detail ensures that players are well taken care of both on and off the field, fostering a positive and productive environment where athletes can thrive.\n\nThis collection embodies Ra\n<end_of_turn>\n<start_of_turn>model\n"
    ], return_tensors="pt").to("cuda")


outputs = lora_model.generate(**inputs, max_new_tokens=50, use_cache=True)
tokenizer.batch_decode(outputs)

In [ ]:
outputs

In [ ]:
merged_model = lora_model.merge_and_unload()

In [ ]:
outputs = merged_model.generate(**inputs, max_new_tokens=50, use_cache=True)
tokenizer.batch_decode(outputs)

In [ ]:
tokenizer.decode(outputs[0])

In [ ]:
tokenizer.batch_decode(outputs)

In [ ]:
inputs = tokenizer(
    [
        test_data[0]['text'].split('<start_of_turn>model\n')[
            0] + '<start_of_turn>model\n'
    ], return_tensors="pt").to("cuda")

outputs = base_model.generate(**inputs, max_new_tokens=100, use_cache=True)

In [ ]:
tokenizer.batch_decode(outputs)